<a href="https://colab.research.google.com/github/mtwenzel/teaching/blob/master/05 Probability PPMI Classification Prob Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Legal Notice {display-mode:'form'}
#@markdown Copyright 2019 (Modifications/adaptions) by Fraunhofer MEVIS, Bremen/Hamburg

#@markdown Original Code: Copyright 2018 The TensorFlow Probability Authors.

#@markdown The original code is licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.

#@markdown You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

#@markdown Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

#@markdown Parts of the code have been adapted from https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/images.ipynb

In [2]:
#@title Imports and data download. {display-mode:'form'}
# Trains a Bayesian neural network to classify DAT scan images.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import warnings
import pathlib
import random
import numpy as np
from urllib.request import urlopen
from zipfile import ZipFile

# Dependency imports
from absl import flags

import matplotlib
matplotlib.use("Agg")
%matplotlib notebook
from matplotlib import figure  # pylint: disable=g-import-not-at-top
from matplotlib.backends import backend_agg

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.layers import Input,Dense,GlobalAveragePooling2D,Flatten,concatenate,BatchNormalization, Dropout
from tensorflow.keras.applications import InceptionV3,DenseNet121
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# For TF2, eager execution will break some code, so disable for now.
tf.compat.v1.disable_eager_execution()
warnings.simplefilter(action="ignore")

try:
  import seaborn as sns  # pylint: disable=g-import-not-at-top
  HAS_SEABORN = True
except ImportError:
  HAS_SEABORN = False

tfd = tfp.distributions

zipurl = 'https://github.com/mtwenzel/parkinson-classification/raw/master/data/PPMI-classification.zip'
zipresp = urlopen(zipurl)
tempzip = open("PPMI-classification.zip", "wb")
tempzip.write(zipresp.read())
tempzip.close()
print("download complete, extracting...")

zf = ZipFile("PPMI-classification.zip")
zf.extractall(path = 'data/')
zf.close()
print("... done")

download complete, extracting...
... done


In [0]:
#@title Parameters {display-mode:'form', run:'auto'}

# PPMI
IMAGE_SHAPE = [109, 91]
num_classes = 2

learning_rate = 0.001
max_steps = 2000 #@param {type:'integer'}
batch_size = 128 #@param {type:'integer'}
data_dir  = os.path.join(os.getenv("TEST_TMPDIR", "/tmp"), "ppmi_basemodel_bnn/data")
model_dir = os.path.join(os.getenv("TEST_TMPDIR", "/tmp"), "ppmi_basemodel_bnn_original/")
viz_steps = 250 #@param {type:'integer'}
num_monte_carlo = 50 #@param {type:'integer'}

In [0]:
#@title Function definitions {display-mode:'form'}
#@markdown Plotting of validation images and weight distributions.
def plot_weight_posteriors(names, qm_vals, qs_vals, fname):
  """Save a PNG plot with histograms of weight means and stddevs.

  Args:
    names: A Python `iterable` of `str` variable names.
    qm_vals: A Python `iterable`, the same length as `names`,
      whose elements are Numpy `array`s, of any shape, containing
      posterior means of weight varibles.
    qs_vals: A Python `iterable`, the same length as `names`,
      whose elements are Numpy `array`s, of any shape, containing
      posterior standard deviations of weight varibles.
    fname: Python `str` filename to save the plot to.
  """
  fig = figure.Figure(figsize=(6, 3))
  canvas = backend_agg.FigureCanvasAgg(fig)

  ax = fig.add_subplot(1, 2, 1)
  for n, qm in zip(names, qm_vals):
    sns.distplot(qm.flatten(), ax=ax, label=n)
  ax.set_title("weight means")
  ax.set_xlim([-1.5, 1.5])
  ax.legend()

  ax = fig.add_subplot(1, 2, 2)
  for n, qs in zip(names, qs_vals):
    sns.distplot(qs.flatten(), ax=ax)
  ax.set_title("weight stddevs")
  ax.set_xlim([0, 1.])

  fig.tight_layout()
  canvas.print_figure(fname, format="png")
  print("saved {}".format(fname))


def plot_heldout_prediction(input_vals, probs,
                            fname, n=10, offset=0, title=""):
  """Save a PNG plot visualizing posterior uncertainty on heldout data.

  Args:
    input_vals: A `float`-like Numpy `array` of shape
      `[num_heldout] + IMAGE_SHAPE`, containing heldout input images.
    probs: A `float`-like Numpy array of shape `[num_monte_carlo,
      num_heldout, num_classes]` containing Monte Carlo samples of
      class probabilities for each heldout sample.
    fname: Python `str` filename to save the plot to.
    n: Python `int` number of datapoints to vizualize.
    offset: Python 'int' offset into the data if not wishing to visualize first 'n' examples
    title: Python `str` title for the plot.
  """
  fig = figure.Figure(figsize=(9, 3*n))
  canvas = backend_agg.FigureCanvasAgg(fig)
  for i in range(offset, offset+n):
    ax = fig.add_subplot(n, 3, 3*(i-offset) + 1)
    ax.imshow(input_vals[i, :].reshape(IMAGE_SHAPE), interpolation="None")

    ax = fig.add_subplot(n, 3, 3*(i-offset) + 2)
    for prob_sample in probs:
      sns.barplot(np.arange(num_classes), prob_sample[i, :], alpha=0.1, ax=ax)
      ax.set_ylim([0, 1])
    ax.set_title("posterior samples")

    ax = fig.add_subplot(n, 3, 3*(i-offset) + 3)
    sns.barplot(np.arange(num_classes), np.mean(probs[:, i, :], axis=0), ax=ax)
    ax.set_ylim([0, 1])
    ax.set_title("predictive probs")
  fig.suptitle(title)
  fig.tight_layout()

  canvas.print_figure(fname, format="png")
  print("saved {}".format(fname))

In [10]:
#@title Check GPU availability {display-mode: 'form'}
#@markdown If you don't see a GPU in Colab, activate it by selecting *Runtime -> Change Runtime Type* in the menu. You can inspect details with code commented out below.

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

#print('')
#print('Detailed information:')
#print('---------------------')
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

Found GPU at: /device:GPU:0


In [0]:
#@title Set the data generators. {display-mode:'form', run: "auto"}
#@markdown Currently not used!

#@markdown Data augmentation choices. Cell runs automatically if anything is changed.
shear_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
zoom_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
width_shift_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
height_shift_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
rotation_range = 10 #@param {type:"slider", min:0, max:90, step:5}
horizontal_flip = True #@param {type:"boolean"}
vertical_flip = False #@param {type:"boolean"}
#@markdown Data source (No need to change if the download succeeded.)
data_directory = '/content/data/PPMI-classification/' #@param ['z:/Data/Parkinson_DATScans UKE/full_ppmi_data/png/', '/content/drive/My Drive/MEVIS/Data/PPMI-classification/'] {allow-input: true}

## Create a tf.data.Dataset from disk

In [0]:
#@title Function definitions and parameters {display-mode:"form"}
#@markdown Set the data paths here, if they are different in your setting.
# This uses code from the TensorFlow tutorials at https://www.tensorflow.org/tutorials/load_data/images

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_path = '/content/data/PPMI-classification/all_2d_train/' #@param {type:"string"}
valid_path = '/content/data/PPMI-classification/all_2d_val/'  #@param {type:"string"}
model_dir = '/content/tmp/test/' #@param {type:"string"}
download_dir = '/content/tmp/test/' #@param {type:"string"}
train_data_root = pathlib.Path(train_path) 
valid_data_root = pathlib.Path(valid_path) 

def preprocess_image(image):
  image = tf.image.decode_png(image,channels=1)
  image = tf.image.resize(image, IMAGE_SHAPE)
  image /= 255.0  # normalize to [0,1] range

  return image

def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)

def get_dataset(data_root):
  all_image_paths = list(data_root.glob('*/*'))
  all_image_paths = [str(path) for path in all_image_paths]
  random.shuffle(all_image_paths)

  image_count = len(all_image_paths)
  print(data_root, image_count)
  label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
  label_to_index = dict((name, index) for index,name in enumerate(label_names))
  all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                      for path in all_image_paths]

  path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
  image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
  label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
  image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
  return image_label_ds, image_count

def build_input_pipeline(training_dataset, heldout_dataset, batch_size, train_size, heldout_size):
    """Build an Iterator switching between train and heldout data."""

    # Build an iterator over training batches.
    training_batches = training_dataset.shuffle(train_size, reshuffle_each_iteration=True).repeat().batch(batch_size)
    training_iterator = tf.compat.v1.data.make_one_shot_iterator(training_batches)

    # Build a iterator over the heldout set with batch_size=heldout_size,
    # i.e., return the entire heldout set as a constant.
    heldout_frozen = (heldout_dataset.take(heldout_size).repeat().batch(heldout_size))
    heldout_iterator = tf.compat.v1.data.make_one_shot_iterator(heldout_frozen)

    # Combine these into a feedable iterator that can switch between training
    # and validation inputs.
    handle = tf.compat.v1.placeholder(tf.string, shape=[])
    feedable_iterator = tf.compat.v1.data.Iterator.from_string_handle(
        handle, 
        tf.compat.v1.data.get_output_types(training_batches), 
        tf.compat.v1.data.get_output_shapes(training_batches))
    images, labels = feedable_iterator.get_next()

    return images, labels, handle, training_iterator, heldout_iterator


In [13]:
if tf.io.gfile.exists(model_dir):
    tf.compat.v1.logging.warning("Warning: deleting old log directory at {}".format(model_dir))
    tf.io.gfile.rmtree(model_dir)
tf.io.gfile.makedirs(model_dir)

train_data, num_train_data = get_dataset(train_data_root)
valid_data, num_valid_data = get_dataset(valid_data_root)

(images, labels, handle, training_iterator, heldout_iterator) = build_input_pipeline(train_data, valid_data, batch_size, num_train_data, num_valid_data)

/content/data/PPMI-classification/all_2d_train 1097
/content/data/PPMI-classification/all_2d_val 193


## Create Model

We use Flipout for linear variance reduction in the weights as proposed in https://arxiv.org/abs/1803.04386. 

In [9]:
# Custom network with all probabilistic layers
with tf.compat.v1.name_scope("bayesian_neural_net", values=[images]):
    neural_net = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tfp.layers.Convolution2DFlipout(64, kernel_size=3, activation=tf.nn.relu),
#        tf.keras.layers.BatchNormalization(),
#        tfp.layers.Convolution2DFlipout(64, kernel_size=3, activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=[2, 2]),
        tf.keras.layers.BatchNormalization(),
        tfp.layers.Convolution2DFlipout(64, kernel_size=3, activation=tf.nn.relu),
#        tf.keras.layers.BatchNormalization(),
#        tfp.layers.Convolution2DFlipout(64, kernel_size=3, activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=[2, 2]),
        tf.keras.layers.BatchNormalization(),
        tfp.layers.Convolution2DFlipout(96, kernel_size=3, activation=tf.nn.relu),
#        tf.keras.layers.BatchNormalization(),
#        tfp.layers.Convolution2DFlipout(96, kernel_size=3, activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=[2, 2]),
        tf.keras.layers.BatchNormalization(),
        tfp.layers.Convolution2DFlipout(128, kernel_size=3, activation=tf.nn.relu),
#        tf.keras.layers.BatchNormalization(),
#        tfp.layers.Convolution2DFlipout(128, kernel_size=3, activation=tf.nn.relu),
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tfp.layers.DenseFlipout(128, activation=tf.nn.relu),
        tfp.layers.DenseFlipout(num_classes)
    ])

    logits = neural_net(images)
    labels_distribution = tfd.Categorical(logits=logits)

neural_net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_flipout (Conv2DFlipou multiple                  1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  256       
_________________________________________________________________
conv2d_flipout_1 (Conv2DFlip multiple                  73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  2

In [0]:
# Compute the -ELBO as the loss, averaged over the batch size.
neg_log_likelihood = -tf.reduce_mean(input_tensor=labels_distribution.log_prob(labels))
kl = sum(neural_net.losses) / num_train_data
elbo_loss = neg_log_likelihood + kl

# Build metrics for evaluation. Predictions are formed from a single forward
# pass of the probabilistic layers. They are cheap but noisy predictions.
predictions = tf.argmax(input=logits, axis=1)
accuracy, accuracy_update_op = tf.compat.v1.metrics.accuracy(labels=labels, predictions=predictions)

# Extract weight posterior statistics for layers with weight distributions
# for later visualization.
names = []
qmeans = []
qstds = []
for i, layer in enumerate(neural_net.layers):
    try:
        q = layer.kernel_posterior
    except AttributeError:
        continue
    names.append("Layer {}".format(i))
    qmeans.append(q.mean())
    qstds.append(q.stddev())

with tf.compat.v1.name_scope("train"):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(elbo_loss)

init_op = tf.group(tf.compat.v1.global_variables_initializer(), tf.compat.v1.local_variables_initializer())


## Train and evaluate the model

In [0]:
# From https://arxiv.org/pdf/1812.03973.pdf:
# Alternatively, run the following instead of a manual train_op.
neural_net.compile(optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate), loss=’categorical_crossentropy’, metrics=[’accuracy’])
neural_net.fit(features, labels, batch_size=32, epochs=5)

# When converting classification TFP code from TF1.x to TF2.x, 
# replacing the KLD in the ELBO may be done with the tfp.layers.KLDivergenceAddLoss layer, just using standard cross-entropy loss.


In [11]:
best_loss = np.inf
best_loss_iter = 0
best_weights = neural_net.get_weights()

with tf.compat.v1.Session() as sess:
    sess.run(init_op)
    # Run the training loop.
    train_handle = sess.run(training_iterator.string_handle())
    heldout_handle = sess.run(heldout_iterator.string_handle())
    for step in range(max_steps):
        _ = sess.run([train_op, accuracy_update_op], feed_dict={handle: train_handle})

        if step % 100 == 0:
            loss_value, accuracy_value = sess.run([elbo_loss, accuracy], feed_dict={handle: train_handle})

            # save best model
            if loss_value < best_loss:
                print("Saving new best model at Step: {:>5d} Loss: {:.4f} Accuracy: {:.4f}".format(step, loss_value, accuracy_value))
                best_loss = loss_value
                best_loss_iter = step
                best_weights = neural_net.get_weights()
            else:
                print("Step: {:>5d} Loss: {:.4f} Accuracy: {:.4f}".format(step, loss_value, accuracy_value))


        if (step+1) % viz_steps == 0:
            # Compute log prob of heldout set by averaging draws from the model:
            # p(heldout | train) = int_model p(heldout|model) p(model|train)
            #                   ~= 1/n * sum_{i=1}^n p(heldout | model_i)
            # where model_i is a draw from the posterior p(model|train).
            probs = np.asarray([sess.run((labels_distribution.probs),
                                     feed_dict={handle: heldout_handle})
                            for _ in range(num_monte_carlo)])
            mean_probs = np.mean(probs, axis=0)

            image_vals, label_vals = sess.run((images, labels), feed_dict={handle: heldout_handle})
            heldout_lp = np.mean(np.log(mean_probs[np.arange(mean_probs.shape[0]), label_vals.flatten()]))
            print(" ... Held-out nats: {:.4f}".format(heldout_lp))

            qm_vals, qs_vals = sess.run((qmeans, qstds))

            if HAS_SEABORN:
                plot_weight_posteriors(names, qm_vals, qs_vals,
                                 fname=os.path.join(
                                     model_dir,
                                     "step{:05d}_weights.png".format(step)))

                plot_heldout_prediction(image_vals, probs, n=20, offset=55,
                                  fname=os.path.join(
                                      model_dir,
                                      "step{:05d}_pred.png".format(step)),
                                  title="mean heldout logprob {:.2f}"
                                  .format(heldout_lp))
                
    # Evaluate best model
    print("Evaluating best model from iter {:>5d}:".format(best_loss_iter))
    neural_net.set_weights(best_weights)

    heldout_handle = sess.run(heldout_iterator.string_handle())
    image_vals, label_vals = sess.run((images, labels), feed_dict={handle: heldout_handle})
    heldout_lp = np.mean(np.log(mean_probs[np.arange(mean_probs.shape[0]), label_vals.flatten()]))

    print(" ... Held-out nats: {:.4f}".format(heldout_lp)) 
    
    qm_vals, qs_vals = sess.run((qmeans, qstds))

    if HAS_SEABORN:
        plot_weight_posteriors(names, qm_vals, qs_vals,
                         fname=os.path.join(
                             model_dir,
                             "best_step{:05d}_weights.png".format(best_loss_iter)))

        plot_heldout_prediction(image_vals, probs, n=20, offset=55,
                          fname=os.path.join(
                              model_dir,
                              "best_step{:05d}_pred.png".format(best_loss_iter)),
                          title="mean heldout logprob {:.2f}"
                          .format(heldout_lp))

Saving new best model at Step:     0 Loss: 2864.0652 Accuracy: 0.6641
Saving new best model at Step:   100 Loss: 2740.0776 Accuracy: 0.7191
Saving new best model at Step:   200 Loss: 2630.2827 Accuracy: 0.8211
 ... Held-out nats: -0.0953
saved /content/tmp/test/step00249_weights.png
saved /content/tmp/test/step00249_pred.png
Saving new best model at Step:   300 Loss: 2521.4866 Accuracy: 0.8654
Saving new best model at Step:   400 Loss: 2413.3696 Accuracy: 0.8892
 ... Held-out nats: -0.0656
saved /content/tmp/test/step00499_weights.png
saved /content/tmp/test/step00499_pred.png
Saving new best model at Step:   500 Loss: 2306.1060 Accuracy: 0.9050
Saving new best model at Step:   600 Loss: 2199.8037 Accuracy: 0.9153
Saving new best model at Step:   700 Loss: 2094.2632 Accuracy: 0.9226
 ... Held-out nats: -0.0725
saved /content/tmp/test/step00749_weights.png
saved /content/tmp/test/step00749_pred.png
Saving new best model at Step:   800 Loss: 1989.8492 Accuracy: 0.9281
Saving new best mod

In [12]:
from google.colab import drive
drive.mount('/content/user_drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/user_drive


In [0]:
# Store the model temporarily into the Colab Files area
neural_net.save('/content/user_drive/My Drive/MEVIS/Projects/DeepLearning/ParkinsonClassificationSPECT/tmp/test/best_weights.h5')
